In [19]:
%%writefile fetch_data.py
from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import os
import re
import json
import selenium

# Initialize Chrome Selenium
options = webdriver.ChromeOptions()
options.add_argument('--no-sandbox')
options.add_argument('--headless')
options.add_argument('--ignore-certificate-errors')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--disable-extensions')
options.add_argument('--disable-gpu')
# user_agent = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.50 Safari/537.36'
user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537'
options.add_argument('user-agent={0}'.format(user_agent))

driver = webdriver.Chrome(options=options) 
# driver = webdriver.Chrome(executable_path=ChromeDriverManager().install(), options=options) 
# driver.implicitly_wait(90)
driver.set_page_load_timeout(90)

# Load the URL and get the page source
URL = 'https://liveat678bellflower.com/floor-plans.aspx'
driver.get(URL)

results_container = WebDriverWait(driver, 100).until(
    EC.presence_of_element_located((By.ID, 'par_10617247'))
)

assert type(results_container) == selenium.webdriver.remote.webelement.WebElement, "results_container is a wrong type! Please check."

sections = results_container.find_elements(By.CLASS_NAME, "unit-container")

# Find elements
res = []
for idx, section in enumerate(sections):
    unit_type_str = section.find_element(By.XPATH, '//*[@id="text-area"]/div/div[1]/h2').get_attribute("innerHTML").strip()
    unit_price_str = section.find_element(By.CLASS_NAME, "unit-rent").get_attribute("innerHTML")
    unit_number_str = section.find_element(By.CLASS_NAME, "unit-number").get_attribute("innerHTML")
    unit_sqft_str = section.find_element(By.CLASS_NAME, "unit-sqft").get_attribute("innerHTML")

    unit_sqft = int(re.findall(r'\d+', unit_sqft_str.replace(',', ''))[0])
    unit_number = int(re.findall(r'\d+', unit_number_str)[0])
    unit_price = unit_price_str.replace('$', '').replace(',', '')
    unit_price = int(unit_price[-4:])
    
    assert unit_type_str == 'The Preserve', 'Wrong unit type! Please check.'
    
    _res = {"unit_number": unit_number,
            "unit_price": unit_price,
            "unit_sqft": unit_sqft
            }
    
    res.append(_res)

assert len(res) != 0, "Result list is empty! Please check."

res2 = sorted(res, key=lambda x: (x['unit_price'], x['unit_number']))

# Write out results
with open("unit_prices.json", "w") as f:
    # Dump the list to the file
    json.dump(res2, f)


Overwriting fetch_data.py


### Get Bellflower apartment page

In [21]:
len(sections)

4

In [18]:
# requests.get(URL).text


In [13]:
from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import os
import re
import json

# Initialize Chrome Selenium
options = webdriver.ChromeOptions()
options.add_argument('--no-sandbox')
options.add_argument('--headless')
options.add_argument('--ignore-certificate-errors')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--disable-extensions')
options.add_argument('--disable-gpu')
# user_agent = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.50 Safari/537.36'
user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537'
options.add_argument('user-agent={0}'.format(user_agent))

driver = webdriver.Chrome(options=options) 
# driver = webdriver.Chrome(executable_path=ChromeDriverManager().install(), options=options) 
# driver.set_page_load_timeout(90)
driver.implicitly_wait(90)

In [14]:
URL = 'https://liveat678bellflower.com/floor-plans.aspx'

# Load the URL and get the page source
driver.get(URL)



In [15]:
results_container = WebDriverWait(driver, 100).until(
    # EC.presence_of_element_located((By.CLASS_NAME, "par-units"))
    EC.presence_of_element_located((By.ID, 'par_10617247'))
)

sections = results_container.find_elements(By.CLASS_NAME, "unit-container")

In [39]:
# section.find_element(By.CSS_SELECTOR, "div.unit-container-left > div:nth-child(3)").get_attribute("innerHTML")
section.find_element(By.CSS_SELECTOR, "div.unit-container-left > div:nth-child(4)").get_attribute("innerHTML").replace('Lease Term: ', '')

'12'

'04/06/2024'

In [37]:
# Find elements
res = []
for idx, section in enumerate(sections):
    unit_type_str = section.find_element(By.XPATH, '//*[@id="text-area"]/div/div[1]/h2').get_attribute("innerHTML").strip()
    unit_price_str = section.find_element(By.CLASS_NAME, "unit-rent").get_attribute("innerHTML")
    unit_number_str = section.find_element(By.CLASS_NAME, "unit-number").get_attribute("innerHTML")
    unit_sqft_str = section.find_element(By.CLASS_NAME, "unit-sqft").get_attribute("innerHTML")
    available_date_str = section.find_element(By.CSS_SELECTOR, "div.unit-container-left > div:nth-child(3)").get_attribute("innerHTML")
    lease_term_str = section.find_element(By.CSS_SELECTOR, "div.unit-container-left > div:nth-child(4)").get_attribute("innerHTML")

    unit_sqft = int(re.findall(r'\d+', unit_sqft_str.replace(',', ''))[0])
    unit_number = int(re.findall(r'\d+', unit_number_str)[0])
    unit_price = unit_price_str.replace('$', '').replace(',', '')
    unit_price = int(unit_price[-4:])
    available_date = re.findall(r'\d{2}/\d{2}/\d{4}', 'Available: 04/06/2024')[0]
    lease_term = int(lease_term_str.replace('Lease Term: ', ''))
    
    assert unit_type_str == 'The Preserve', 'Wrong unit type! Please check.'
    
    _res = {"unit_number": unit_number,
            "unit_price": unit_price,
            "unit_sqft": unit_sqft,
            "available_date": available_date,
            "lease_term": lease_term
            }
    
    res.append(_res)
    # res.append(f"{unit_type_str} is renting for ${unit_price:,} ({unit_number_str}, {unit_sqft_str})")
    # print(f"{unit_type_str}: {unit_number_str} is renting for ${unit_price:,} ({unit_sqft_str})")

res2 = sorted(res, key=lambda x: (x['unit_price'], x['unit_number']))
res2    

[{'unit_number': 2,
  'unit_price': 3403,
  'unit_sqft': 1076,
  'available_date': '04/06/2024'},
 {'unit_number': 58,
  'unit_price': 3403,
  'unit_sqft': 1076,
  'available_date': '04/06/2024'},
 {'unit_number': 88,
  'unit_price': 3419,
  'unit_sqft': 1076,
  'available_date': '04/06/2024'},
 {'unit_number': 91,
  'unit_price': 3529,
  'unit_sqft': 1076,
  'available_date': '04/06/2024'}]

In [189]:
import json

res2

with open("my_list.json", "w") as f:
    # Dump the list to the file
    json.dump(res2, f)

### Discord bot

## Tesla

### Fetch Tesla's page
[Selenium getting started](https://selenium-python.readthedocs.io/getting-started.html)  
[Locating elements](https://selenium-python.readthedocs.io/locating-elements.html#locating-elements)

* The _Keys_ class provide keys in the keyboard like RETURN, F1, ALT etc. 
* The _By_ class is used to locate elements within a document.

In [85]:
# TESLA_URL = 'https://www.tesla.com/en_CA/inventory/new/my?TRIM=LRAWD&PAINT=BLACK&ADL_OPTS=TOWING&arrangeby=plh&zip=h4r0j1&range=200'
TESLA_URL = 'https://www.tesla.com/inventory/new/my?TRIM=LRAWD&WHEELS=NINETEEN&CABIN_CONFIG=FIVE&arrangeby=plh&zip=94043&range=0'

# Load the URL and get the page source
driver.get(TESLA_URL)

results_container = WebDriverWait(driver, 100).until(
    EC.presence_of_element_located((By.CLASS_NAME, "results-container"))
)

car_sections = results_container.find_elements(By.CLASS_NAME, "result-header")


### Filter HTML

In [74]:
car_sections[-1].find_element(By.CLASS_NAME, "result-price-base-price").get_attribute("innerHTML")
# car_section.find_element(By.CLASS_NAME, "result-purchase-price").get_attribute("innerHTML")
# <span class="tds-text--caption result-price-base-price">$47,990</span>

'$50,990'

In [86]:
car_prices = []

# Now you can iterate over these article elements and perform any actions you desire.
for idx, car_section in enumerate(car_sections):

    car_price_str = car_section.find_element(By.CLASS_NAME, "result-purchase-price").get_attribute("innerHTML")
    # car_basic_info = car_section.find_element(By.CLASS_NAME, "tds-text_color--10").get_attribute("innerHTML")
    try: 
        car_base_price_str = car_section.find_element(By.CLASS_NAME, "result-price-base-price").get_attribute("innerHTML")
    except Exception as e:
        car_base_price_str = car_price_str
    # car_price_str = '1234'

    car_price = int(car_price_str.replace('$', '').replace(',', ''))
    car_base_price = int(car_base_price_str.replace('$', '').replace(',', ''))
    pct = (car_base_price - car_price) / car_base_price * 100
    # print(str(idx) + ' ' + str(car_price) + ' | ' + str(car_base_price))
    car_prices.append(str(car_price) + ' a ' + str(pct) + ' difference!')


### Print output

In [87]:
for price in set(car_prices):
    # if price < 73290:
    print(f'There is a model Y for sale for {price}!')

There is a model Y for sale for 46070 a 4.000833506980621 difference!!
There is a model Y for sale for 48490 a 3.000600120024005 difference!!
There is a model Y for sale for 47990 a 0.0 difference!!
There is a model Y for sale for 46540 a 5.001020616452338 difference!!
There is a model Y for sale for 48440 a 5.000980584428319 difference!!
There is a model Y for sale for 47990 a 4.000800160032006 difference!!
There is a model Y for sale for 47510 a 1.0002083767451553 difference!!
There is a model Y for sale for 45590 a 5.001041883725776 difference!!
There is a model Y for sale for 47490 a 5.001000200040008 difference!!
There is a model Y for sale for 47030 a 4.00081649316187 difference!!
There is a model Y for sale for 47520 a 3.0006123698714022 difference!!
There is a model Y for sale for 47030 a 2.0004167534903106 difference!!
There is a model Y for sale for 46550 a 3.000625130235466 difference!!
There is a model Y for sale for 48010 a 2.000408246580935 difference!!


In [88]:
import requests

url = "https://www.tesla.com/inventory/api/v1/inventory-results?query={%22query%22:{%22model%22:%22my%22,%22condition%22:%22new%22,%22options%22:{%22TRIM%22:[%22LRAWD%22],%22AUTOPILOT%22:[%22AUTOPILOT%22]},%22arrangeby%22:%22Price%22,%22order%22:%22asc%22,%22market%22:%22US%22,%22language%22:%22en%22,%22super_region%22:%22north%20america%22,%22lng%22:-122.1257,%22lat%22:47.6722,%22zip%22:%2294401%22,%22range%22:100},%22offset%22:0,%22count%22:50,%22outsideOffset%22:0,%22outsideSearch%22:false}"
resp = requests.get(url, timeout=30)

ReadTimeout: HTTPSConnectionPool(host='www.tesla.com', port=443): Read timed out. (read timeout=30)